In [1]:
import sys
import os

import rasterio
import geopandas as gpd
import osmnx as ox

# project lib
PROJECT_SRC_PATH = os.path.join(os.path.abspath(''), '..')
sys.path.append(PROJECT_SRC_PATH)

import util

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
GHS_built_up_path = os.path.join(PROJECT_SRC_PATH, "data", "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R4_C19.tif")
topo_path = os.path.join(PROJECT_SRC_PATH, "data", "gmted2010-mea075.tif")
GHS_pop_path = os.path.join(PROJECT_SRC_PATH, "data", "GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0.tif")

corine_lu_path = os.path.join(PROJECT_SRC_PATH, "data", "U2018_CLC2018_V2020_20u1.gpkg")
oceans_path = os.path.join(PROJECT_SRC_PATH, "data", "OSM-water-polygons-split.gpkg")

cdd_path = os.path.join(PROJECT_SRC_PATH, "data", "CDD_historical_mean_v1.nc")
hdd_path = os.path.join(PROJECT_SRC_PATH, "data", "HDD_historical_mean_v1.nc")

city_dir = os.path.join(PROJECT_SRC_PATH, "tests", "data", "bldgs")
dir_out = os.path.join(PROJECT_SRC_PATH, "tests", "data")
buildings = util.load_buildings(city_dir, "Vaugneray")


In [20]:
# Raster .tif data

for file, file_out in [(topo_path, 'GMTED_topography_test_region.tif'), (GHS_pop_path, 'GHS_POP_test_region.tif'), (GHS_built_up_path, 'GHS_BUILT_test_region.tif')]:
    path_out = os.path.join(dir_out, file_out)

    area = util.bbox(buildings, buffer=10000)
    raster, meta = util.read_area(file, area)
    profile = rasterio.open(file).profile
    profile |= meta

    with rasterio.open(path_out, "w", **profile) as dest:
        dest.write(raster)

In [21]:
# Land use

box = util.bbox(buildings, crs="EPSG:3035", buffer=10000)
lu = gpd.read_file(corine_lu_path, bbox=box)
lu.to_file(os.path.join(dir_out, "CORINE_landuse_test_region.gpkg"), driver="GPKG")

/usr/local/Caskroom/miniconda/base/envs/eubucco-features/lib/python3.10/site-packages/geopandas/io/file.py:484: UserWarning: More than one layer found in 'U2018_CLC2018_V2020_20u1.gpkg': 'U2018_CLC2018_V2020_20u1' (default), 'U2018_CLC2018_V2020_20u1_FR_REU', 'U2018_CLC2018_V2020_20u1_FR_GLP', 'U2018_CLC2018_V2020_20u1_FR_GUF', 'U2018_CLC2018_V2020_20u1_FR_MTQ', 'U2018_CLC2018_V2020_20u1_FR_MYT'. Specify layer parameter to avoid this warning.
  crs = pyogrio.read_info(path_or_bytes).get("crs")
/usr/local/Caskroom/miniconda/base/envs/eubucco-features/lib/python3.10/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'U2018_CLC2018_V2020_20u1.gpkg': 'U2018_CLC2018_V2020_20u1' (default), 'U2018_CLC2018_V2020_20u1_FR_REU', 'U2018_CLC2018_V2020_20u1_FR_GLP', 'U2018_CLC2018_V2020_20u1_FR_GUF', 'U2018_CLC2018_V2020_20u1_FR_MTQ', 'U2018_CLC2018_V2020_20u1_FR_MYT'. Specify layer parameter to avoid this warning.
  result = read_func(


In [ ]:
# OCEANS / COAST

box = util.bbox(buildings, crs="EPSG:3857", buffer=1e6)
oceans = gpd.read_file(oceans_path, bbox=box)
oceans.to_file(os.path.join(dir_out, "OSM_oceans_test_region.gpkg"), driver="GPKG")

In [ ]:
# POIs

box = util.bbox(buildings, crs="EPSG:3857", buffer=5000)
pois = poi.download(box)
pois.to_file(os.path.join(dir_out, "Vaugneray_pois.gpkg"), driver="GPKG")

In [ ]:
# OSM buildings

ox.config(timeout=1000)
tags = {'building': True}

box = util.bbox(buildings, crs="EPSG:4326")
east, south, west, north = box.total_bounds
osm_bldgs = ox.geometries.geometries_from_bbox(north, south, east, west, tags)
# osm_bldgs[['geometry', 'height', 'building']].to_file('../tests/data/Vaugneray/Vaugneray_osm_bldgs.gpkg', driver='GPKG')
osm_bldgs['old_geometry'] = osm_bldgs['geometry']
osm_bldgs['type_source'] = osm_bldgs['building']
osm_bldgs['geometry'] = osm_bldgs['geometry'].apply(lambda x: x.wkt)
osm_bldgs[['geometry', 'height', 'type_source']].to_csv(os.path.join(dir_out, "Vaugneray_osm_bldgs.csv"), index=False)